In [78]:
from bs4 import BeautifulSoup as bs
import re
from dateutil.parser import parse
# from datetime import datetime
import pandas as pd

In [140]:
def parse_html(path): 
    with open(path) as f:
        # read file
        content = f.read()
        # parse with bs
        soup = bs(content, 'lxml')

    return soup

In [141]:
watch_history = parse_html('../Rådata/Takeout/YouTube and YouTube Music/history/watch-history.html')

In [142]:
def is_a_video(tag): 
    '''
    Filter to remove ads and videos that still ex
    takes a block of html code and returns a boolean with whether this is an ad or not
    '''
    not_an_ad = not(bool(re.search('From Google Ads', tag.get_text())))
    not_deleted = len(tag.find_all('a')) > 2
    return (not_an_ad & not_deleted)
       
 

In [143]:
def parse_watch_history(soup):
    '''takes a parsed tree of watch history (wh) from beautiful soup and returns a simple df with the relevant data'''
    # hver yderste blok har klassen 'outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp' - Her henter vi dem alle sammen
    blocks = soup.find_all(class_ = 'outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp')
    blocks_cleaned = [block for block in blocks if is_a_video(block)]
    print(f'{len(blocks)} entries; {len(blocks_cleaned)} after cleaning. {len(blocks) - len(blocks_cleaned)} ads and deleted videos removed!')
    return blocks_cleaned

watch_history_blocks = parse_watch_history(watch_history)

30345 entries; 28332 after cleaning. 2013 ads and deleted videos removed!


In [137]:
# Finding a video and an add for testing! 
# video = watch_history_blocks[0]
# ad =  watch_history_blocks[5]

# To create the dataset! 

In [139]:
watch_history_blocks[0]

<div class="outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp"><div class="mdl-grid"><div class="header-cell mdl-cell mdl-cell--12-col"><p class="mdl-typography--title">YouTube<br/></p></div><div class="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1">Watched <br/><br/>9 Aug 2022, 13:27:58 CEST</div><div class="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1 mdl-typography--text-right"></div><div class="content-cell mdl-cell mdl-cell--12-col mdl-typography--caption"><b>Products:</b><br/> YouTube<br/><b>Why is this here?</b><br/> This activity was saved to your Google Account because the following settings were on: YouTube watch history. You can control these settings  .</div></div></div>

In [145]:
video_titles = []
video_links = []
channel_titles = []
channel_links = []
date_watched = []

i = 0
for block in watch_history_blocks:
    links = block.find_all('a')
    
    video_titles.append(links[0].text)
    video_links.append(links[0]['href'])
    channel_titles.append(links[1].text)
    channel_links.append(links[1]['href'])
    i += 1
    print(f'{len(links)} links; entry {i} with title {links[0].text}')
    # problem med den her regex - hvis en youtube kanal har et tal i slutningen af sit navn så bliver det taget med og vi kan få forkerte
    # eller i værste fald nonsens-datoer. En løsning kunne være at fjerne alle links inden vi tager teksten
    # forsøg med at slette links! 
    for link in links:
        link.decompose()

    date_string = re.search(r'\d?\d \w{3, 4} \d{4}, \d{2}:\d{2}:\d{2}[^P]*', block.text)
    date_watched.append(parse(date_string[0]))


3 links; entry 1 with title Schrödinger's Levels - Astortion Devlog
3 links; entry 2 with title John mayer - neon (after 2 years of practice)
3 links; entry 3 with title Are Chinese watches worth it?
3 links; entry 4 with title An Intricate $180 Beauty! | The Sugess Moonphase Master
3 links; entry 5 with title Corgeut Seamaster 300 Spectre v Omega Seamaster 300 Spectre
3 links; entry 6 with title Corgeut "Spectre" 007 (Seamaster 300 Homage Review)
3 links; entry 7 with title Stranger Things / High school Musical
3 links; entry 8 with title I bought an unusual telecaster.
3 links; entry 9 with title Ni No Kuni: Wrath of the White Witch Switch Review
3 links; entry 10 with title YES, I BECAME A KARTHUS ENJOYER!
3 links; entry 11 with title Nathan For You: Asian Accent, Valet Issue
3 links; entry 12 with title Bob's Burgers - Bacon Burger
3 links; entry 13 with title THE Godfather of Pizza
3 links; entry 14 with title Dining at a 3 Michelin Star Restaurant in Korea
3 links; entry 15 with 

TypeError: 'NoneType' object is not subscriptable

In [107]:
date_watched[:200]

[<re.Match object; span=(66, 91), match='9 Aug 2022, 13:27:58 CEST'>,
 <re.Match object; span=(69, 94), match='9 Aug 2022, 13:27:39 CEST'>,
 <re.Match object; span=(62, 87), match='9 Aug 2022, 13:13:38 CEST'>,
 <re.Match object; span=(74, 99), match='9 Aug 2022, 09:46:47 CEST'>,
 <re.Match object; span=(91, 116), match='9 Aug 2022, 09:45:38 CEST'>,
 <re.Match object; span=(82, 107), match='9 Aug 2022, 09:29:18 CEST'>,
 <re.Match object; span=(65, 90), match='8 Aug 2022, 23:04:20 CEST'>,
 <re.Match object; span=(57, 82), match='8 Aug 2022, 18:31:38 CEST'>,
 <re.Match object; span=(78, 103), match='7 Aug 2022, 21:23:00 CEST'>,
 <re.Match object; span=(57, 82), match='7 Aug 2022, 20:58:13 CEST'>,
 <re.Match object; span=(62, 88), match='77 Aug 2022, 20:57:57 CEST'>,
 <re.Match object; span=(48, 73), match='7 Aug 2022, 16:26:33 CEST'>,
 <re.Match object; span=(47, 72), match='7 Aug 2022, 14:26:29 CEST'>,
 <re.Match object; span=(72, 97), match='7 Aug 2022, 14:25:32 CEST'>,
 <re.Match objec

In [120]:
links = video.find_all('a')
links[0]['href']
for link in links:
    link.decompose()
links

[<None></None>]

In [121]:
video

<div class="outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp"><div class="mdl-grid"><div class="header-cell mdl-cell mdl-cell--12-col"><p class="mdl-typography--title">YouTube<br/></p></div><div class="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1">Watched <br/><br/>9 Aug 2022, 13:27:58 CEST</div><div class="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1 mdl-typography--text-right"></div><div class="content-cell mdl-cell mdl-cell--12-col mdl-typography--caption"><b>Products:</b><br/> YouTube<br/><b>Why is this here?</b><br/> This activity was saved to your Google Account because the following settings were on: YouTube watch history. You can control these settings  .</div></div></div>

In [99]:
date_string = re.search(r'\d?\d \w{3} \d{4}, \d{2}:\d{2}:\d{2}[^P]*', video.text)


In [101]:
date = parse(date_string[0])
date_string[0]

'9 Aug 2022, 13:27:58 CEST'

In [84]:
date.isoformat()

'2022-08-09T13:27:58+02:00'